In [322]:
import tweepy as tw
import pandas as pd
import numpy as np
import time
import re
import sys
import os

In [323]:
os.chdir('/Users/gk/Desktop/Northeastern/MSDS/DS5230_Unsupervised_Machine_Learning/Project/Twitter-Network-Analysis/Notebooks/')

In [227]:
auth = tw.OAuthHandler('4CmtKtsKT0gzw3iuSJB8LN9nV', 'xDP2YlcAyRkC4jhyshM3q1UHN9bp3AYe2QGvyxtHszmBdqclsM')

In [228]:
auth.set_access_token('65785693-zUuHedvCJc1xIBiZiOfxvdBKk211hZpvRR6XLyh1r', '98H4uN9y4HEsldiyJzCfejUUdQqJxC811c2RCxxnmrzuv')

In [229]:
api = tw.API(auth, wait_on_rate_limit=True)

## Hashtag Pipeline

#### Find and Filter tweets

In [230]:
search_words = "#COVID19"
date_since = "2020-03-11"
num_items = 1000

new_search = search_words + " -filter:retweets"

tweets = tw.Cursor(api.search,
              q=new_search,
              lang="en",
              since=date_since).items(num_items)

#### Get hashtags into a dataframe

In [231]:
start_time = time.time()
hash_tags = [[tweet.id, tweet.entities['hashtags']] for tweet in tweets if tweet.entities['hashtags'] != []]
end_time = time.time()
duration = end_time - start_time
duration

108.32331109046936

In [232]:
result = []
for T in hash_tags:
    for d in T[1]:
        result.append([T[0], d["text"]])

result_df = pd.DataFrame(data=result, columns=['tweet_id','hashtag'])

#### Build a collection of tweets with a minimum number of hashtags and join with the initial results

In [233]:
covid_tweets = result_df.groupby('tweet_id').count()

In [234]:
good_tweets = covid_tweets[covid_tweets['hashtag'] > 2]

In [235]:
good_results = pd.merge(result_df, good_tweets, on='tweet_id', how='outer').dropna()
good_results = good_results[good_results['hashtag_x'] != search_words.replace("#","")]
good_results = good_results[good_results['hashtag_x'] != "COVID"]

good_results = good_results[(~good_results['hashtag_x'].str.contains("COVID"))\
                            & (~good_results['hashtag_x'].str.contains("covid"))\
                           & (~good_results['hashtag_x'].str.contains("Covid"))\
                           &(~good_results['hashtag_x'].str.contains("CORONA"))\
                           &(~good_results['hashtag_x'].str.contains("corona"))\
                           &(~good_results['hashtag_x'].str.contains("Corona"))] 

In [236]:
good_results.nunique() #hashtag_x is unique hashtags, hashtag_y is unique counts of hashtags

tweet_id     162
hashtag_x    356
hashtag_y      9
dtype: int64

In [237]:
#test = good_results.groupby(['tweet_id','hashtag_x']).size().reset_index().rename(columns={0:'count'})
#test[test['count'] > 1]

good_results_unique = good_results.drop_duplicates(['tweet_id','hashtag_x'])

#### CSV

In [238]:
good_results_unique.to_csv(r'/Users/gk/Desktop/Northeastern/MSDS/DS5230_Unsupervised_Machine_Learning/Project/COVID19_hashtags_TEST.csv')

## Experimentation

#### Tweet and User attributes

In [239]:
#tweet.text
#tweet.user.friends_count

#attributes = [[tweet.user.screen_name, tweet.user.friends_count, tweet.text] for tweet in tweets]

In [240]:
#tweet_text = pd.DataFrame(data=attributes, columns=['user', "friends_count", "text"])

In [241]:
#tweet_text['text'][0]

#### Fix messed up hashtag_y values

In [316]:
fix_hashtag_y = pd.read_csv('/Users/gk/Desktop/Northeastern/MSDS/DS5230_Unsupervised_Machine_Learning/Project/csv_archive/COVID19_hashtags7.csv')

In [317]:
hashtag_y_df = fix_hashtag_y['tweet_id'].value_counts().to_frame()
test = pd.merge(fix_hashtag_y, hashtag_y_df, how='left', left_on=['tweet_id'], right_on=[hashtag_y_df.index])
test = test.iloc[: , [1,2,3,5]]
test = test.rename(columns={'tweet_id_y': 'hashtag_y'})

In [319]:
test.nunique()

Unnamed: 0    6008
tweet_id_x    2579
hashtag_x     3509
hashtag_y       11
dtype: int64

In [318]:
test.to_csv(r'/Users/gk/Desktop/Northeastern/MSDS/DS5230_Unsupervised_Machine_Learning/Project/csv_archive/COVID19_hashtags7_update.csv',index=False)

#### Remove URLs

In [246]:
def remove_url(txt):
    """Replace URLs found in a text string with nothing 
    (i.e. it will remove the URL from the string).

    Parameters
    ----------
    txt : string
        A text string that you want to parse and remove urls.

    Returns
    -------
    The same txt string with url's removed.
    """

    return " ".join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", txt).split())

In [243]:
#tweet_text['text'] = [remove_url(tweet) for tweet in tweets2[tweets2.columns[2]]]
#tweet_text['text'][0]

In [271]:
#test = remove_duplicates7.groupby(['tweet_id','hashtag_x']).size().reset_index().rename(columns={0:'count'})
#test[test['count'] > 1]

#remove_duplicates7 = pd.read_csv(r'/Users/gk/Desktop/test_COVID19_hashtags7.csv')
#remove_duplicates7 = remove_duplicates7.drop_duplicates(['tweet_id','hashtag_x'])
#remove_duplicates7[["tweet_id"]].to_string()

In [264]:
#remove_duplicates7.to_csv(r'/Users/gk/Desktop/COVID19_hashtags7.csv', index=False)

In [265]:
start_time = time.time()
tweet_text = []
for tweet_id in good_results_unique['tweet_id'].unique():
    try:
        text = api.get_status(tweet_id, tweet_mode = 'extended')
    except tw.TweepError:
        continue
    tweet_text.append([tweet_id, text.full_text])
    
tweet_text_df = pd.DataFrame(data=tweet_text, columns=['tweet_id','text']).drop_duplicates(['text'], keep='first')
end_time = time.time()
duration = end_time - start_time
duration

2130.8475909233093

In [272]:
#duplicate_latest = tweet_text_df.groupby(['text']).size().reset_index().rename(columns={0:'count'})
#duplicate_latest[duplicate_latest['count'] > 1]

In [269]:
tweet_text_df.to_csv(r'/Users/gk/Desktop/COVID19_hashtags7_text_TEST.csv')

#### Fix Text File

In [48]:
#text_data = pd.read_csv('/Users/gk/Desktop/Northeastern/MSDS/DS5230_Unsupervised_Machine_Learning/Project/twitter-network-graph/Hashtag_Data/COVID19_hashtags5_text.csv')

In [51]:
#test_text = text_data.groupby(['tweet_id','text']).size().reset_index().rename(columns={0:'count'})
#test_text[test_text['count'] > 1]

#error_tweets = text_data[(text_data['text'] == 'Error')]['tweet_id']
#text_data[text_data['tweet_id'].isin(error_tweets)]

In [224]:
#duplicate_text = text_data.groupby(['text']).size().reset_index().rename(columns={0:'count'})
#duplicate_text[duplicate_text['count'] > 1]

#text_data[text_data['text'].str.contains('#BREAKING Kentucky reports first death due to')]

#### Find Friends

In [22]:
user = api.get_user('geoffkorb')

In [23]:
def limit_handled(cursor):
    while True:
        try:
            yield cursor.next()
        except tw.RateLimitError:
            time.sleep(15 * 60)

gk_friends = []            
            
for friend in limit_handled(tw.Cursor(api.friends).items()):
    print(friend.screen_name)

HamelHusain
NaturelsLit
JonnyKimUSA
aruneshmathur
davidgoggins
nickfirchau
haxor
vboykis
NewAmericaPIT
hunterwalk
ProjectLincoln
davidrberke
jockowillink
soccer_minute
AndrewM_Webb
matt_levine
SolidSoundFest
ashk4n
lexfridman
luis_likes_math
random_walker
numberphile
Kurz_Gesagt
justmarkham
Artoftheproblem
PostalService
geoffreyhinton
seb_ruder
GoogleAI
tristanharris
3blue1brown
AeronautBrewing
MultilayerNets
fastdotai
j6aer
SpotifyEng
GustavS
DebateVis
MIT_CSAIL
costaleabo
waitbutwhy
lflockwood
erez_kaminski
policingproject
TheBostonCal
lucyparsonslabs
jeremyphoward
themarkup
chipro
DanOUrban
kdnuggets
carvellwallace
btenergy
indexingai
OpenAI
math3ma
brookLYNevery1
olgavitek
kuwisdelu
stefanjwojcik
W4ngatang
hugobowne
nu4good
EveryNoise
_brohrer_
therriaultphd
DevotedHealth
StanfordHAI
AndrewYNg
NU_PolicySchool
NUlabTMN
AllenDowney
drivendataorg
albarrentine
gourmetboutique
nugradunion
bibim_box
VolunteerSci
jovialjoy
MforJ
rahulbot
SCDAONews
GoNUmhockey
IvyChild
CathyClassical
DARol

RuntimeError: generator raised StopIteration

In [ ]:
#edgelist (followers followees)
#attribute (user attribute)

In [ ]:
# import csv and run nunique()